<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 5.1.1
# *Evaluating Classification Models*

In this lab you will be practicing evaluating classification models (Logistic Regression in particular) on a "Breast Cancer" data.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs

%matplotlib inline

### 1. Load Data

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. n the 3-dimensional space is that described in: [K. P. Bennett and O. L. Mangasarian: "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].

This database is also available through the UW CS ftp server: ftp ftp.cs.wisc.edu cd math-prog/cpo-dataset/machine-learn/WDBC/

Also can be found on UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Attribute Information:

1) ID number 2) Diagnosis (M = malignant, B = benign) 3-32)

Ten real-valued features are computed for each cell nucleus:

    a) radius (mean of distances from center to points on the perimeter) 
    b) texture (standard deviation of gray-scale values) 
    c) perimeter 
    d) area 
    e) smoothness (local variation in radius lengths) 
    f) compactness (perimeter^2 / area - 1.0) 
    g) concavity (severity of concave portions of the contour) 
    h) concave points (number of concave portions of the contour) 
    i) symmetry 
    j) fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

Missing attribute values: none

Class distribution: 357 benign, 212 malignant

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report

In [6]:
breast_cancer_csv = ('breast-cancer-wisconsin-data.csv')
breast = pd.read_csv(breast_cancer_csv)

### 2. EDA 

Explore dataset. Check for nulls. Make the target column binary.

In [7]:
breast.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [8]:
breast.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [9]:
breast.set_index('id', inplace=True)

breast.drop('Unnamed: 32', axis=1, inplace=True)
breast.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [10]:
breast = pd.get_dummies(breast)
breast.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_B,diagnosis_M
id,,,,,,,,,,,,,,,,,,,,,
842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,1
842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,1
84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,1
84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,1
84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,1


In [11]:
breast = breast.drop('diagnosis_B', axis=1)
breast.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_M
id,,,,,,,,,,,,,,,,,,,,,
842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [12]:
breast.dtypes

radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst            float64
concave points_worst       float64
symmetry_worst      

In [13]:
X = breast[[c for c in breast.columns if c != 'diagnosis_M']]
y = breast['diagnosis_M']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [14]:
model_coef_sorted = np.sort(abs(model.coef_))[0]
for col in zip(X.columns, model_coef_sorted):
    print('{:<10s} {:+.06f}'.format(col[0], col[1]))

radius_mean +0.011174
texture_mean +0.014628
perimeter_mean +0.014756
area_mean  +0.024119
smoothness_mean +0.029945
compactness_mean +0.032420
concavity_mean +0.041187
concave points_mean +0.041702
symmetry_mean +0.043991
fractal_dimension_mean +0.047233
radius_se  +0.048045
texture_se +0.082999
perimeter_se +0.113747
area_se    +0.126283
smoothness_se +0.141480
compactness_se +0.147448
concavity_se +0.154955
concave points_se +0.210604
symmetry_se +0.236851
fractal_dimension_se +0.259219
radius_worst +0.331511
texture_worst +0.366978
perimeter_worst +0.412367
area_worst +0.420608
smoothness_worst +0.438004
compactness_worst +0.465128
concavity_worst +0.475508
concave points_worst +0.510189
symmetry_worst +0.747672
fractal_dimension_worst +0.962302


In [15]:
from sklearn.feature_selection import SelectKBest

In [16]:
select = SelectKBest(k=10)
select.fit(X, y)
select.scores_
features = pd.DataFrame(data = [X.columns, select.scores_]).T
features.columns = ['features', 'score']
features.sort_values(ascending=False, by = 'score').iloc[:7, : ]

,features,score
27,concave points_worst,964.385
22,perimeter_worst,897.944
7,concave points_mean,861.676
20,radius_worst,860.782
2,perimeter_mean,697.235
23,area_worst,661.6
0,radius_mean,646.981


### 3. Logistic Regression Model

#### 3.1 Use Logistic Regression

Use Logistic Regression and examine accuracy score, confusion matrix, classification report for that model.

In [31]:
X = breast[['concave points_worst', 'perimeter_worst', 'concave points_mean', 'radius_worst']]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [33]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.9252747252747253

In [34]:
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.8947368421052632

In [35]:
print(metrics.confusion_matrix(y_test, y_pred))
print(np.asarray([['TN', 'FP'], ['FN', 'TP']]))

[[70  2]
 [10 32]]
[['TN' 'FP']
 ['FN' 'TP']]


#### 3.2 Calculate the predicted labels and predicted probabilities on the test set.

In [36]:
# predict test class:
y_pred_class = model.predict(X_test)

# predicted test probability:
y_pred_prob = model.predict_proba(X_test)[:,1]



#### 3.3 Create the confusion matrix for your classfier's performance on the test set.

Convert confusion_matrix to a dataframe. 

    index=['is_cancer', 'is_healthy']
    columns=['predicted_cancer','predicted_healthy']

In [37]:
# ANSWER
confusion = metrics.confusion_matrix(y_test, y_pred)

confusion_df = pd.DataFrame(confusion, index=['is cancer', 'is_healthy'], columns=['predicted_cancer', 'predicted_healthy'])

print(confusion_df, '\n')
print(np.asarray([['TN', 'FP'], ['FN', 'TP']]))

            predicted_cancer  predicted_healthy
is cancer                 70                  2
is_healthy                10                 32 

[['TN' 'FP']
 ['FN' 'TP']]


In [38]:
TN = 71
FP = 1
FN = 5
TP = 37

#### 3.4 Calculate Accuracy

**Classification Accuracy:** Overall, how often is the classifier correct?
    
    (TP + TN) / float(TP + TN + FP + FN)

In [39]:
# ANSWER
# use float to perform true division, not integer division
accuracy = metrics.accuracy_score(y_test, y_pred)
assert accuracy == (TP + TN) / float(TP + TN + FP + FN)
print(accuracy)

AssertionError: 

#### 3.5 Calculate Classification Error

**Classification Error:** Overall, how often is the classifier incorrect?

Also known as "Misclassification Rate"

    (FP + FN) / float(TP + TN + FP + FN)

In [40]:
# ANSWER
assert 1 - metrics.accuracy_score == (FP + FN) / float(TP + TN + FP + FN)

TypeError: unsupported operand type(s) for -: 'int' and 'function'

#### 3.6 Calculate Sensitivity

**Sensitivity:** When the actual value is positive, how often is the prediction correct?

Something we want to maximize
How "sensitive" is the classifier to detecting positive instances?
- Also known as "True Positive Rate" or "Recall"
- TP / all positive
    - all positive = TP + FN
    
    sensitivity = TP / float(FN + TP)

In [41]:
# ANSWER
print(classification_report(y_test, y_pred, output_dict=False))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92        72
           1       0.94      0.76      0.84        42

    accuracy                           0.89       114
   macro avg       0.91      0.87      0.88       114
weighted avg       0.90      0.89      0.89       114



In [42]:
classification = classification_report(y_test, y_pred, output_dict=True)
sensitivity = metrics.recall_score(y_test, y_pred)
assert sensitivity == TP / float(FN + TP)
print(classification)

AssertionError: 

#### 3.7 Calculate Specificity

**Specificity:** When the actual value is negative, how often is the prediction correct?

- Something we want to maximize. How "specific" (or "selective") is the classifier in predicting positive instances?
- TN / all negative
    - all negative = TN + FP
    
    specificity = TN / (TN + FP)

In [50]:
# ANSWER
specificity = classification['0']['recall']
assert specificity == TN / (TN + FP)

AssertionError: 

In [51]:
specifity = classification['0']['recall']
assert specifity == TN/ (TN + FP)
print(specifity)

AssertionError: 

#### 3.8 Calculate False Positive Rate

**False Positive Rate:** When the actual value is negative, how often is the prediction incorrect?

    false_positive_rate = FP / float(TN + FP)

In [29]:
# ANSWER
fpr, _, _ = metrics.roc_curve(y_test, y_pred)
assert fpr[1] == FP / (TN + FP)
print(fpr)

AssertionError: 

#### 3.9 Calculate Precision

**Precision:** When a positive value is predicted, how often is the prediction correct?

How "precise" is the classifier when predicting positive instances?

    precision = TP / float(TP + FP)

In [30]:
# ANSWER
classification = classification_report(y_test, y_pred, output_dict=True)
precision = classification['1']['precision']
assert precision == TP / float (TP + FP)

print(precision)

AssertionError: 

#### 3.10 Create a predicted test probability dataframe with column name 'class_0_pp','class_1_pp'

In [96]:
# ANSWER
# Get the predicted probability vector and explicitly name the columns:)
y_pred_prob = model.predict_proba(X_test)
y_pred_v = pd.DataFrame(y_pred_prob, columns = ['class_0_pp', 'class_1_pp'])
y_pred_v

,class_0_pp,class_1_pp
0,0.426998,0.573002
1,0.319805,0.680195
2,0.998890,0.001110
3,0.008826,0.991174
4,0.167681,0.832319
...,...,...
109,0.007391,0.992609
110,0.388543,0.611457
111,0.999915,0.000085
112,0.635959,0.364041


#### 3.11 The Receiver operating characteristic (ROC) curve

Plot the ROC curve for Class 1

In [44]:
from sklearn.metrics import roc_curve, auc

In [45]:
# For class 1, find the area under the curve
# Find fpr, tpr
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_v['class_1_pp'])
# Find auc
roc_auc = metrics.auc(fpr, tpr)
# Plot of a ROC curve for class 1 (has_cancer)
plt.figure(figsize=[8,8])
plt.plot(fpr, tpr, color='darkorange', lw=4, label = 'ROC curve (area = %0.2f)' % roc_auc)
# Plot fpr, tpr
plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic for cancer detection', fontsize=18)
plt.legend(loc="lower right")
plt.show()

NameError: name 'y_pred_v' is not defined

**References**

[Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/downloads/breast-cancer-wisconsin-data.zip/2)

[Validation curves: plotting scores to evaluate models](https://scikit-learn.org/stable/modules/learning_curve.html#learning-curves)



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



